In [1]:
# Initialise
%run ~/relmapping/annot/notebooks/__init__.ipynb
def vp(fp): return os.path.join('annot/S2_regulatory_annotation/metrics/', fp) # "verbose path"

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/b2/scratch/ahringer/jj374/lab/relmapping


In [2]:
# Load hypersensitive sites
df_sites = pd.read_csv('annot/S1_accessible_sites/S1a_accessible_sites.tsv', sep='\t')[['chrom', 'start', 'end', 'atac_mode']]
#df_sites = df_sites.head(100)
print(len(df_sites), 'ATAC-seq sites')

42245 ATAC-seq sites


In [3]:
# Jump test

# 2-condition, 2-replicate one-sided test
def deseq2x2(df_counts):
    fh_inp, fp_inp = tempfile.mkstemp(dir=os.getcwd(), prefix='deseq2x2_inp_')
    fh_out, fp_out = tempfile.mkstemp(dir=os.getcwd(), prefix='deseq2x2_out_')
    df_counts.to_csv(fp_inp, sep='\t')
    !cat {fp_inp} | scripts/deseq2x2_greater.R > {fp_out}
    #!wc -l {fp_inp}
    #!wc -l {fp_out}
    #!tail -n 20 {fp_out}
    df_out = pd.read_csv(fp_out, sep='\s+')
    assert len(df_counts) == len(df_out)
    !rm {fp_inp}
    !rm {fp_out}
    return df_out

# Count reads & call DESeq
def jump_test(fp_rep1, fp_rep2, chrom, u_start, u_end, d_start, d_end, f):
    df_counts = pd.DataFrame()
    df_counts['ucount_rep1'] = list(map(f, yp.read_regions(fp_rep1, chrom, u_start, u_end)))
    df_counts['ucount_rep2'] = list(map(f, yp.read_regions(fp_rep2, chrom, u_start, u_end)))
    df_counts['dcount_rep1'] = list(map(f, yp.read_regions(fp_rep1, chrom, d_start, d_end)))
    df_counts['dcount_rep2'] = list(map(f, yp.read_regions(fp_rep2, chrom, d_start, d_end)))
    df_deseq = deseq2x2(df_counts)
    return pd.concat([df_counts, df_deseq], axis=1)

def itemRgb_(passed):
    if passed:
        return yp.RED
    else:
        return yp.BLUE

def jump_test_fwd(stage):
    step = 'trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.filled_fwd'
    fp_rep1 = pf('lcap808_%(stage)s_rep1' % locals(), step, '.bw', 'lcap808')
    fp_rep2 = pf('lcap808_%(stage)s_rep2' % locals(), step, '.bw', 'lcap808')
    chrom = df_sites['chrom'].tolist()
    u_start = (df_sites['atac_mode'] - 250).tolist()
    u_end = (df_sites['atac_mode'] - 75 + 1).tolist()
    d_start = (df_sites['atac_mode'] + 75).tolist()
    d_end = (df_sites['atac_mode'] + 250 + 1).tolist()
    df_jump = jump_test(fp_rep1, fp_rep2, chrom=chrom, u_start=u_start, u_end=u_end,
                        d_start=d_start, d_end=d_end, f=lambda s: int(np.nansum(s)))
    df_jump['passed'] = (df_jump['log2FoldChange'] > 2) & (df_jump['padj'] < 0.05)
    df_jump['summary'] = ['%s / %.02f / %.3g' % (passed, lfc, padj) \
                          for passed, lfc, padj in zip(df_jump['passed'], df_jump['log2FoldChange'], df_jump['padj'])]
    df_jump.columns = ['lcap_%s_fwd_%s' % (stage, col) for col in df_jump.columns]
    
    write_gffbed(vp('lcap_%s_fwd.bed' % (stage,)),
        chrom = chrom,
        start = u_start,
        thickStart = np.array(u_end) - 1,
        thickEnd = np.array(d_start) + 1,
        end = d_end,
        attr = df_jump,
        itemRgb = list(map(itemRgb_, df_jump['lcap_%s_fwd_passed' % (stage,)]
    )))
    return df_jump

df_jump_fwd = pd.concat(pmap(jump_test_fwd, config['stages'], n_jobs=15), axis=1)
df_jump_fwd.to_csv(vp('lcap_all_fwd.tsv'), sep='\t', index=False)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
mean-dispersion relationship
mean-dispersion relationship
final dispersion estimates
mean-dispersion relationship

[Parallel(n_jobs=15)]: Done  11 out of  11 | elapsed:  1.8min finished


In [4]:
def jump_test_rev(stage):
    step = 'trim20.bwa_pe.rm_unmapped_pe.rm_chrM.rm_rRNA_broad.rm_blacklist.rm_q10.filled_rev'
    fp_rep1 = pf('lcap808_%(stage)s_rep1' % locals(), step, '.bw', 'lcap808')
    fp_rep2 = pf('lcap808_%(stage)s_rep2' % locals(), step, '.bw', 'lcap808')
    chrom = df_sites['chrom'].tolist()
    d_start = (df_sites['atac_mode'] - 250).tolist()
    d_end = (df_sites['atac_mode'] - 75 + 1).tolist()
    u_start = (df_sites['atac_mode'] + 75).tolist()
    u_end = (df_sites['atac_mode'] + 250 + 1).tolist()
    df_jump = jump_test(fp_rep1, fp_rep2, chrom=chrom, u_start=u_start, u_end=u_end,
                        d_start=d_start, d_end=d_end, f=lambda s: int(np.nansum(s)))
    df_jump['passed'] = (df_jump['log2FoldChange'] > 2) & (df_jump['padj'] < 0.05)
    df_jump['summary'] = ['%s / %.02f / %.3g' % (passed, lfc, padj) \
                          for passed, lfc, padj in zip(df_jump['passed'], df_jump['log2FoldChange'], df_jump['padj'])]
    df_jump.columns = ['lcap_%s_rev_%s' % (stage, col) for col in df_jump.columns]
    
    write_gffbed(vp('lcap_%s_rev.bed' % (stage,)),
        chrom = chrom,
        start = d_start,
        thickStart = np.array(d_end) - 1,
        thickEnd = np.array(u_start) + 1,
        end = u_end,
        attr = df_jump,
        itemRgb = list(map(itemRgb_, df_jump['lcap_%s_rev_passed' % (stage,)]
    )))
    return df_jump

df_jump_rev = pd.concat(pmap(jump_test_rev, config['stages'], n_jobs=15), axis=1)
df_jump_rev.to_csv(vp('lcap_all_rev.tsv'), sep='\t', index=False)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating size factors
estimating dispersions
estimating dispersions
gene-wise dispersion estimates
estimating size factors
gene-wise dispersion estimates
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating dispersions
gene-wise dispersion estimates
estimating dispersions
gene-wise dispersion estimates
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
mean-dispersion relationship
mean-dispersion relationship
mean-dispersion relationship
final dispersion estimates

[Parallel(n_jobs=15)]: Done  11 out of  11 | elapsed:  1.9min finished


In [5]:
pd.concat([df_jump_fwd['lcap_%s_fwd_passed' % (stage,)].value_counts() for stage in config['stages']], axis=1)

,lcap_wt_emb_fwd_passed,lcap_wt_l1_fwd_passed,lcap_wt_l2_fwd_passed,lcap_wt_l3_fwd_passed,lcap_wt_l4_fwd_passed,lcap_wt_ya_fwd_passed,lcap_glp1_d1_fwd_passed,lcap_glp1_d2_fwd_passed,lcap_glp1_d6_fwd_passed,lcap_glp1_d9_fwd_passed,lcap_glp1_d13_fwd_passed
False,36530,37075,36640,36676,36628,36641,38930,39220,38418,38903,38370
True,5715,5170,5605,5569,5617,5604,3315,3025,3827,3342,3875


In [6]:
pd.concat([df_jump_rev['lcap_%s_rev_passed' % (stage,)].value_counts() for stage in config['stages']], axis=1)

,lcap_wt_emb_rev_passed,lcap_wt_l1_rev_passed,lcap_wt_l2_rev_passed,lcap_wt_l3_rev_passed,lcap_wt_l4_rev_passed,lcap_wt_ya_rev_passed,lcap_glp1_d1_rev_passed,lcap_glp1_d2_rev_passed,lcap_glp1_d6_rev_passed,lcap_glp1_d9_rev_passed,lcap_glp1_d13_rev_passed
False,36584,37062,36512,36733,36641,36783,38932,39385,38539,38904,38452
True,5661,5183,5733,5512,5604,5462,3313,2860,3706,3341,3793


In [7]:
u_start = (df_sites['atac_mode'] - 250).tolist()
u_end = (df_sites['atac_mode'] - 75 + 1).tolist()
d_start = (df_sites['atac_mode'] + 75).tolist()
d_end = (df_sites['atac_mode'] + 250 + 1).tolist()

l_passed_any = df_jump_fwd[['lcap_%s_fwd_passed' % (stage,) for stage in config['stages']]].any(axis=1)
write_gffbed(vp('lcap_all_fwd.bed'),
    chrom = df_sites['chrom'],
    start = u_start,
    thickStart = np.array(u_end) - 1,
    thickEnd = np.array(d_start) + 1,
    end = d_end,
    attr = df_jump_fwd[['lcap_%s_fwd_summary' % (stage,) for stage in config['stages']]],
    itemRgb = list(map(itemRgb_, l_passed_any)),
)

l_passed_any = df_jump_rev[['lcap_%s_rev_passed' % (stage,) for stage in config['stages']]].any(axis=1)
write_gffbed(vp('lcap_all_rev.bed'),
    chrom = df_sites['chrom'],
    start = u_start,
    thickStart = np.array(u_end) - 1,
    thickEnd = np.array(d_start) + 1,
    end = d_end,
    attr = df_jump_rev[['lcap_%s_rev_summary' % (stage,) for stage in config['stages']]],
    itemRgb = list(map(itemRgb_, l_passed_any)),
)